2 CNN: 0.98
1 CNN: 0.9615 epoch 3
1 CNN: 0.9782 epoch 50
ABC: 0.9575 batch 100, lr 0.0001, metrics criterion 'CE', epoch 3
ABC: 0.9671 batch 100, lr 0.0001, metrics criterion 'CE', epoch 6
ABC: 0.9768 epoch 50 kernel_per_pixel 8

In [1]:
import torch
import torchvision
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.tools import dotdict
from driver.driver import ABC_Driver
from torch_geometric_temporal import METRLADatasetLoader
from other_model.other_model import make_default_model
import atd2022
torch.cuda.is_available()
torch.cuda.set_device(0)

In [2]:
atd_args = dotdict()

atd_args.name = 'atd'
atd_args.train_batch_size = 25
atd_args.predict_len = 4
atd_args.history_len = 7

atd_args.train_epochs= 10
atd_args.lr = 0.00009
atd_args.criterion = 'L1'
atd_args.use_gpu = True
atd_args.if_scheduler = False

activation = 'relu'
pool_name = 'avg'
input_channel = 1
input_height = atd_args.history_len
input_width = 5200
pixel_number = input_height*input_width
kernel_size = 6
knpp = 12
knpp2 = atd_args.predict_len

atd_args.layers=[
    ('agnostic', ((input_channel, knpp, kernel_size), 1, pool_name, (1,1), activation)),
    ('agnostic', ((knpp, knpp2, kernel_size), 1, pool_name, (1,1), activation)),
    ('linear', (knpp2*input_height, atd_args.predict_len, (1,2), 1, (1, atd_args.predict_len))),
]

In [3]:
wt_args = dotdict()

wt_args.name = 'wiki_traffic'
wt_args.train_batch_size = 30
wt_args.predict_len = 1
wt_args.history_len = 5

wt_args.train_epochs= 10
wt_args.lr = 0.001
wt_args.criterion = 'L1'
wt_args.use_gpu = True
wt_args.if_scheduler = True

activation = 'relu'
pool_name = 'avg'
input_channel = 1
input_height = wt_args.history_len
input_width = 1400
pixel_number = input_height*input_width
kernel_size = 3
knpp = 5
knpp2 = 15

wt_args.layers=[
    ('specific', ((input_channel, knpp, kernel_size), 1, pool_name, (1,1), activation)),
    ('specific', ((knpp, knpp2, kernel_size), 1, pool_name, (1,1), activation)),
    ('linear', (knpp2*input_height, wt_args.predict_len, (1,2), 1, (1, wt_args.predict_len))),
]

In [4]:
lat_args = dotdict()

lat_args.name = 'lat'
lat_args.train_batch_size = 30
lat_args.predict_len = 1
lat_args.history_len = 5

lat_args.train_epochs= 10
lat_args.lr = 0.001
lat_args.criterion = 'L1'
lat_args.use_gpu = True
lat_args.if_scheduler = True

activation = 'relu'
pool_name = 'avg'
input_channel = 1
input_height = lat_args.history_len
input_width = 207
pixel_number = input_height*input_width
kernel_size = 3
knpp = 5
knpp2 = 15

lat_args.layers=[
    ('specific', ((input_channel, knpp, kernel_size), 1, pool_name, (1,1), activation)),
    ('specific', ((knpp, knpp2, kernel_size), 1, pool_name, (1,1), activation)),
    ('linear', (knpp2*input_height, lat_args.predict_len, (1,2), 1, (1, lat_args.predict_len))),
]

In [17]:
mnist_args = dotdict()

mnist_args.name = 'mnist'
mnist_args.train_batch_size = 60
mnist_args.predict_batch_size = 100

mnist_args.train_epochs = 10
mnist_args.lr = 0.0001
mnist_args.criterion = 'CE'
mnist_args.use_gpu = True
mnist_args.if_scheduler = True

activation = 'relu'
pool_name = 'avg'
input_channel = 1
input_height = 28
input_width = 28
pixel_number = input_height*input_width
kernel_size = 25
knpp = 10
knpp2 = 20
knpp3 = 39

# mnist_args.layers=[
#     ('specific', ((input_channel, knpp, kernel_size), 1, pool_name, (1,1), activation)),
#     ('specific', ((knpp, knpp2, kernel_size), 1, pool_name, (1,1), activation)),
#     ('specific', ((knpp2, knpp3, kernel_size), 1, pool_name, (1,1), activation)),
#     ('attention', (knpp3, 3)),
#     ('linear', (knpp3, 10, (1,2,3))),
#     ('softmax', (1))
# ]

mnist_args.layers=[
    ('cnn2d', ((input_channel, knpp, (5,5), 1, 2), 1, pool_name, (1,1), activation)),
    ('cnn2d', ((knpp, knpp2, (5,5), 1, 2), 1, pool_name, (1,1), activation)),
    ('cnn2d', ((knpp2, knpp3, (5,5), 1, 2), 1, pool_name, (1,1), activation)),
    ('attention', (knpp3, 3)),
    ('linear', (knpp3, 10, (1,2,3))),
    ('softmax', (1))
]

In [18]:
cifar10_args = dotdict()

cifar10_args.name = 'cifar10'
cifar10_args.train_batch_size = 100
cifar10_args.predict_batch_size = 100

cifar10_args.train_epochs = 120
cifar10_args.lr = 0.0001
cifar10_args.criterion = 'CE'
cifar10_args.use_gpu = True
cifar10_args.if_scheduler = False

activation = 'relu'
pool_name = 'avg'
input_channel = 3
input_height = 32
input_width = 32
pixel_number = input_height*input_width
kernel_size = 9
knpp = [63, 63, 129, 129, 255, 255, 513, 513]
# knpp = [64, 64, 128, 128, 256, 256, 512, 512, 1024, 1024]

cifar10_args.layers=[
    ('specific', ((input_channel, knpp[0], kernel_size), 2, pool_name, (1,1), activation)),
    ('agnostic', ((knpp[0], knpp[1], kernel_size), 2, pool_name, (2,2), activation)),
    ('agnostic', ((knpp[1], knpp[2], kernel_size), 2, pool_name, (1,1), activation)),
    ('agnostic', ((knpp[2], knpp[3], kernel_size), 2, pool_name, (2,2), activation)),
    ('agnostic', ((knpp[3], knpp[4], kernel_size), 2, pool_name, (1,1), activation)),
    ('agnostic', ((knpp[4], knpp[5], kernel_size), 2, pool_name, (2,2), activation)),
    ('agnostic', ((knpp[5], knpp[6], kernel_size), 2, pool_name, (1,1), activation)),
    ('agnostic', ((knpp[6], knpp[7], kernel_size), 2, pool_name, (2,2), activation)),
#     ('agnostic', ((knpp[7], knpp[8], kernel_size), 2, pool_name, (1,1), activation)),
#     ('agnostic', ((knpp[8], knpp[9], kernel_size), 2, pool_name, (2,2), activation)),
#     ('attention', (knpp, 5, 4)),
    ('linear', (int(knpp[-1]*pixel_number/(4**4)), 10, (1,2,3))),
    ('softmax', (1))
]

# cifar10_args.layers=[
#     ('large', ((input_channel, 33, (3,3), 7, (1,1)), 2, pool_name, (1,1), activation)),
#     ('large', ((33, 63, (3,3), 7, (1,1)), 2, pool_name, (1,1), activation)),
#     ('large', ((63, 129, (3,3), 7, (1,1)), 2, pool_name, (1,1), activation)),
#     ('large', ((129, 255, (3,3), 7, (1,1)), 2, pool_name, (1,1), activation)),
#     ('linear', (int(33*pixel_number), 10, (1,2,3))),
#     ('softmax', 1)
# ]

# cifar10_args.layers=[
#     ('cnn2d', ((input_channel, knpp[0], (3,3), 1, 1), 2, pool_name, (1,1), activation)),
#     ('cnn2d', ((knpp[0], knpp[1], (3,3), 1, 1), 2, pool_name, (2,2), activation)),
#     ('cnn2d', ((knpp[1], knpp[2], (3,3), 1, 1), 2, pool_name, (1,1), activation)),
#     ('cnn2d', ((knpp[2], knpp[3], (3,3), 1, 1), 2, pool_name, (2,2), activation)),
#     ('cnn2d', ((knpp[3], knpp[4], (3,3), 1, 1), 2, pool_name, (1,1), activation)),
#     ('cnn2d', ((knpp[4], knpp[5], (3,3), 1, 1), 2, pool_name, (2,2), activation)),
#     ('cnn2d', ((knpp[5], knpp[6], (3,3), 1, 1), 2, pool_name, (1,1), activation)),
#     ('cnn2d', ((knpp[6], knpp[7], (3,3), 1, 1), 2, pool_name, (2,2), activation)),
#     ('cnn2d', ((knpp[7], knpp[8], (3,3), 1, 1), 2, pool_name, (1,1), activation)),
#     ('cnn2d', ((knpp[8], knpp[9], (3,3), 1, 1), 2, pool_name, (2,2), activation)),
# #     ('attention', (knpp2, 5, 4)),
#     ('linear', (int(knpp[-1]*pixel_number/64/4/4), 2048, (1,2,3), None, None, 'relu')),
#     ('linear', (2048, 10, (1,))),
#     ('softmax', (1))
# ]

In [19]:
args = mnist_args
data = None
# data = atd2022.io.read_csv()
# data = pd.read_csv('/scratch/mfeng/data/ABC/Wiki_Traffic/filled_selected_train_1.csv', index_col=0, header=[0,1,2,3], parse_dates=True)
# data = pd.read_csv('/scratch/mfeng/data/ABC/LA_Traffic/LA_Traffic.csv', index_col=0)
# data = data.head(200)

driver = ABC_Driver(args, data)
driver.train()

Use GPU: cuda:None
add record: 01/25/2023 12:52
epoch: 0, train_loss: 2.1282, test_metric: 0.6548
epoch: 1, train_loss: 1.686, test_metric: 0.9334
epoch: 2, train_loss: 1.5551, test_metric: 0.9487
epoch: 3, train_loss: 1.5287, test_metric: 0.9575
epoch: 4, train_loss: 1.5176, test_metric: 0.9585
epoch: 5, train_loss: 1.5167, test_metric: 0.9623
epoch: 6, train_loss: 1.5088, test_metric: 0.9654
epoch: 7, train_loss: 1.5032, test_metric: 0.9694
epoch: 8, train_loss: 1.4995, test_metric: 0.9707
epoch: 9, train_loss: 1.4973, test_metric: 0.9708


In [9]:
driver.metric()

In [8]:
# driver.predict()

tensor([[[[ 5.3340e-01,  4.1910e-01,  6.1760e-01,  ...,  3.0458e+00,
            6.0598e+00,  3.8502e-01],
          [-2.1529e-01, -3.5393e-02, -2.7057e-01,  ..., -2.3178e+00,
           -7.7229e+00,  4.9505e-03],
          [-1.9272e-01, -1.1109e-01,  4.6962e-02,  ..., -4.3600e-01,
            5.5248e-02, -1.3636e-01],
          [ 2.6736e-01,  1.6332e-01,  3.1615e-01,  ...,  3.5126e+00,
            7.0238e+00,  8.1232e-02]]]])

In [26]:
driver.metric() # [64, 64, 128, 128, 256, 256, 512, 512, 1024, 1024] + MLP

0.837

In [10]:
driver.metric() # [64, 64, 128, 128, 256, 256, 512, 512, 1024, 1024] + linear

0.8438